In [578]:
import os.path
import sys

import os

# Récupère le chemin d'accès parent de sys.path[0]
parent_dir = os.path.dirname(sys.path[0])

sys.path.append(os.path.abspath(parent_dir))
import importlib
import algorithmes
importlib.reload(algorithmes)
from algorithmes import *
import random
import mesa

## L'implimentation d'un systeme multi agent qui suit le Protocole Ennemie

## Création des Agents

In [580]:
class Recuit_sem(mesa.Agent):
    
    # Création d'agent Recuit simulé 


    def __init__(self, model,parameters,pool_key):
        
        super().__init__(self,model)
        self.parameters = parameters
        self.pool_key = pool_key # le pool qui va etre amélioré par l'algorithme
        

    def execute(self):

        # Tirer une solution aléatoire du Pool et la remplacer par une solution améliorer par Recuit simulé
        solution_index = np.random.randint(len(self.model.Pool[self.pool_key]))
        new_solution, meilleure_valeur , _  = recuit_simule(self.model.Pool[self.pool_key][solution_index],self.parameters['temperature_initiale'] , self.parameters['temperature_finale'],alpha=0.99)
        self.model.Pool[self.pool_key][solution_index] = new_solution
        
            
    def step(self):
        
        self.execute()

In [582]:
class Tabou(mesa.Agent):
    
    # Création d'agent Tabou 

    def __init__(self,  model,parameters,pool_key):
        super().__init__(self,model)
      
        self.parameters = parameters
        self.pool_key = pool_key # le pool qui va etre amélioré par l'algorithme

    def execute(self):

        # Tirer une solution aléatoire du Pool et la remplacer par une solution améliorer par Tabou
        solution_index = np.random.randint(len(self.model.Pool[self.pool_key]))
        new_solution, meilleure_valeur , _ = tabou(self.model.Pool[self.pool_key][solution_index] ,self.parameters['taille_tabou'],self.parameters['max_iterations'],self.parameters['n_voisin'])
        self.model.Pool[self.pool_key][solution_index] = new_solution

            
    def step(self):
        self.execute()

In [663]:
class Genetic(mesa.Agent):
    
    # Création d'agent genetique 

    def __init__(self,  model,parameters,pool_key):

        super().__init__(self,model)
        self.parameters = parameters
        self.pool_key = pool_key

    def execute(self):

        # Prendre le Pool comme une population initial et la remplacer par une population améliorer par l'algo genetique
        new_solution, meilleure_valeur , _ ,population = genetique(self.model.Pool[self.pool_key],self.parameters['taux_mutation'],self.parameters['max_iterations'])
        self.model.Pool[self.pool_key] = copy.deepcopy(population)
        
            
    def step(self):
        self.execute()

In [589]:
def compute_Pool_avg(model):

    # Calculer le cout moyenne des solutions dans le Pool
    couts = [cout(get_route_version2(model.Pool[i], time_window, Q)) for i in range(len(model.Pool))]
    
    return sum(couts)/len(couts)

def compute_Pool_min(model):
    
    # Calculer le cout minimum des solution dans le Pool
    couts = [cout(get_route_version2(model.Pool[i], time_window, Q)) for i in range(len(model.Pool))]

    return min(couts)

def get_the_best_solution(model):

    # Récupérer la meilleur solution dans le Pool
    couts = np.array([cout(get_route_version2(model.Pool[i], time_window, Q)) for i in range(len(model.Pool))])

    return model.Pool[np.argmin(couts)]

In [593]:
def compute_cout_pool1(model):
    
    # Calculer le cout moyenne des solutions dans le Pool
    couts = [cout(get_route_version2(model.Pool['pool1'][i], time_window, Q)) for i in range(len(model.Pool['pool1']))]

    return min(couts)


def compute_cout_pool2(model):
    
    # Calculer le cout moyenne des solutions dans le Pool
    couts = [cout(get_route_version2(model.Pool['pool2'][i], time_window, Q)) for i in range(len(model.Pool['pool2']))]

    return min(couts)

In [596]:
def compute_1(model):
    
    # Calculer le cout moyenne des solutions dans le Pool
    return model.resultats['equipe1']


def compute_2(model):
    
    # Calculer le cout moyenne des solutions dans le Pool

    return model.resultats['equipe2']

In [645]:
## Les paramètres des Algorithmes 
num_clients = 25
parameters = {'rs' : {'temperature_initiale' : 1  , 'temperature_finale' : 1e-6 } , 'tabou' : {'taille_tabou' : nombre_client , 'max_iterations' : 100 , 'n_voisin' : 2} , 'genetic'  : {'taux_mutation' : 0.5 ,'max_iterations' : 10 }}

In [647]:
class SMA_ENNEMIE(mesa.Model):
    
    """Création d'environnement de communication entre les agents"""


    def __init__(self,Pool,parameters):
        
        self.schedule1 = mesa.time.RandomActivation(self)
        self.schedule2 = mesa.time.RandomActivation(self)
        self.Pool = Pool
        self.resultats = {}

        # Instanciation des agents 
        self.optimazers = {'equipe1' : [Recuit_sem(self,parameters['rs'],'pool1'), Genetic(self,parameters['genetic'],'pool1')], 'equipe2' : [Tabou(self,parameters['tabou'],'pool2'), Genetic(self,parameters['genetic'],'pool2')] }
        self.schedule1.add(self.optimazers['equipe1'][0])
        self.schedule2.add(self.optimazers['equipe2'][0])


        ## Récuperer à chaque step la meilleur solution dans le Pool et son cout
        self.datacollector1 = mesa.DataCollector(
            model_reporters={"pool_1": compute_cout_pool1  } )
        
        self.datacollector2 = mesa.DataCollector(
            model_reporters={"pool_2": compute_cout_pool2 } )


    def step1(self):

        
        self.schedule1.step()
        self.resultats['equipe1'] = compute_cout_pool1(self)
        self.datacollector1.collect(self)


    def step2(self):

        self.schedule2.step()
        self.resultats['equipe2'] = compute_cout_pool2(self)
        self.datacollector2.collect(self)


In [656]:
## Création du Pool
POOL = {}
Pool = []
list_client_initial = [i for i in range(num_clients)]
for j in range(2): 
    x  = copy.deepcopy(list_client_initial)
    random.shuffle(x)
    Pool.append(x)
    POOL['pool1'] = Pool
    
Pool = []
for j in range(2): 
    x  = copy.deepcopy(list_client_initial)
    random.shuffle(x)
    Pool.append(x)
    POOL['pool2'] = Pool

In [658]:
sma_env = SMA_ENNEMIE(POOL,parameters)

### Exécution de notre Système multu agent ennemie

In [674]:
# initiation

sma_env = SMA_ENNEMIE(POOL,parameters)

for i in range(5) : 

    sma_env.step1()
    sma_env.step2()

    if i%10 == 0 :

        sma_env.optimazers['equipe1'][1].execute()
        sma_env.optimazers['equipe2'][1].execute()
    

cout_equipe1 = sma_env.resultats['equipe1']
cout_equipe2 = sma_env.resultats['equipe2']

In [669]:
cout_equipe1 , cout_equipe2

(412.3525726762663, 771.9651053877202)

In [675]:
for i in range(10):

    if cout_equipe1 < cout_equipe2 : 

        while cout_equipe1 < cout_equipe2 : 

            for k in range(15) :

                sma_env.step2()
                
                if k%10 == 0 :
                    sma_env.optimazers['equipe2'][1].execute()

                k = k + 1

            cout_equipe2 = sma_env.resultats['equipe2']

            


    if cout_equipe1 > cout_equipe2 : 

        while cout_equipe1 > cout_equipe2 :
            
            for k in range(10) :

                sma_env.optimazers['equipe1'][1].execute()

                k = k + 1

            cout_equipe1 = sma_env.resultats['equipe1']


    

            

KeyboardInterrupt: 